In [1]:
import numpy as np 
import random
import math

In [2]:
Scene1 = [ [ 100, 100 ] , [100, 400], [400,100], [400, 400] , [250,400] ] 
Scene2 = [ [ 100, 700 ] , [100, 400], [400,100], [400, 400] , [250,250] ] 

Scene3 = []

R = np.asarray( [ [ math.cos(30) , -math.sin(30) ], [ math.sin(30) , math.cos(30) ] ] )

for i in range( len(Scene2) ):
    
    S = R@np.asarray(Scene2[i]).T
    Scene3.append(S)
    
Scene2= Scene3 

In [3]:
def GetEdgeProb(x, mean):
    return np.exp( -np.abs(x- mean) )

In [4]:
def GetAllEdgeProb():
    
    Edgeprob = {} 
    
    for i in range(len( Scene1)):
        for j in range(len(Scene2)):
            
            index = (i,j)
            Dist1 =  np.linalg.norm( np.asarray( Scene1[i] ) - np.asarray( Scene1[j]))
            Dist2 =  np.linalg.norm( np.asarray( Scene2[i] ) - np.asarray( Scene2[j]))
            
            Edgeprob[index] = GetEdgeProb( Dist1 , Dist2)
    
    return Edgeprob

In [5]:
E_Prob = GetAllEdgeProb()
print(E_Prob)

{(0, 0): 1.0, (0, 1): 0.9999999999999432, (0, 2): 9.010511273244143e-162, (0, 3): 0.9999999999999432, (0, 4): 4.6008371328886736e-61, (1, 0): 0.9999999999999432, (1, 1): 1.0, (1, 2): 1.0, (1, 3): 1.0, (1, 4): 1.038485243581605e-27, (2, 0): 9.010511273244143e-162, (2, 1): 1.0, (2, 2): 1.0, (2, 3): 1.0, (2, 4): 2.89050940897982e-54, (3, 0): 0.9999999999999432, (3, 1): 1.0, (3, 2): 1.0, (3, 3): 1.0, (3, 4): 1.038485243581605e-27, (4, 0): 4.6008371328886736e-61, (4, 1): 1.038485243581605e-27, (4, 2): 2.89050940897982e-54, (4, 3): 1.038485243581605e-27, (4, 4): 1.0}


In [6]:
Node_Prob_Init = [0.5,0.5,0.5,0.5, 0.5]

In [7]:
for r in range(2000):

    for i in range(len( Scene1)):
        score = 0 
        for j in range(len( Scene2)):

            edge = (i,j)
            edge_prob = E_Prob[edge]
            score += edge_prob*Node_Prob_Init[j]

        Node_Prob_Init[i] = 1/(1 + np.exp(-score))    

In [8]:
Node_Prob_Init

[0.9481767842196662,
 0.9792629759722802,
 0.9481767842196721,
 0.9792629759722802,
 0.6590460684074066]

In [9]:
def UpdateNodeProb1(Node_Probablity , node_index):
    
    A = 1
    B = 1 
    
    for i in range( len(Node_Probablity )):
        
        if( i !=  node_index ):
            
            A *= E_Prob[ (i , node_index) ]
            B *= Node_Probablity[i]
    
    NodeProb = A/B
    Node_Probablity[node_index] = NodeProb
    print( A, B )
    
    return Node_Probablity

In [10]:
def GetNormalization( Node_Probablity ):
    
    NormalizationVal = 0
    
    for i in range(len( Scene1)):
        for j in range(len( Scene2)):
            
            if(i != j ):
                
                index = (i,j)
                Edge_Prob1 = E_Prob[ (i , j) ]
                A = Node_Probablity[i]
                
                NormalizationVal += Edge_Prob1/A
                
    return NormalizationVal/4      

In [11]:
def UpdateNodeProb2(Node_Probablity , node_index):
    
    A = 1
    B = 1 
    Total = 0
    for i in range( len(Node_Probablity )):
        
        if( i !=  node_index ):
            
            A = E_Prob[ (i , node_index) ]
            B = Node_Probablity[i]
            
            Total += A/B
    
    NodeProb = Total/GetNormalization( Node_Probablity ) 
#     print(GetNormalization( Node_Probablity ) )
    Node_Probablity[node_index] = NodeProb
    
    return Node_Probablity

In [12]:
NumIters = 100
Node_Probablity = Node_Prob_Init
for iters in range(NumIters):
    for i in range( len(Node_Probablity )):
        
        Node_Probablity = UpdateNodeProb2(  Node_Probablity , i)

In [13]:
print( Node_Probablity )

[2.9506512910048424e-27, 2.0, 4.191835316288542e-27, 2.0, 4.4188071853953555e-54]
